In [11]:
from needed_files.quantumenvironment import QuantumEnvironment
from needed_files.helper_functions import load_agent_from_yaml_file
from needed_files.ppo import make_train_ppo
from needed_files.q_env_config import q_env_config as gate_q_env_config
# from needed_files.qenvconfig import QEnvConfiguration

import os
import time
import cloudpickle
import pickle

In [15]:
pickle_file_name = 'gate_q_env_config.pickle'
try:
    with open(pickle_file_name, 'wb') as handle:
        cloudpickle.dump(gate_q_env_config, handle, protocol=pickle.HIGHEST_PROTOCOL)
except:
    print('Configuration CANNOT be serialized.')

Configuration CANNOT be serialized.


In [2]:
# type(QEnvConfiguration().q_env_configuration)

In [3]:
from braket.tracking import Tracker
from braket.jobs import hybrid_job, save_job_result
from braket.jobs.metrics import log_metric
from braket.aws import AwsDevice

from braket.aws import AwsSession
from braket.aws import AwsQuantumJob
from braket.jobs.local import LocalQuantumJob

from qiskit_braket_provider import BraketLocalBackend, AWSBraketProvider, AWSBraketBackend, AWSBraketJob

aws_session = AwsSession(default_bucket="amazon-braket-us-west-1-lukasvoss")

In [4]:
def calibrate_gate(num_total_updates):
    print("Job started!!!!!")

    braket_task_costs = Tracker().start()
    ppo_params, network_params  = load_agent_from_yaml_file(file_path='config_yamls/agent_config.yaml')
    agent_config = {**ppo_params, **network_params}
    
    provider = AWSBraketProvider()
    backend = provider.get_backend('SV1')

    # XXX: Uncomment to not get error with non-serializability of gate_q_env_config object,
    #      but then get error in hybrib jobb logs 'ModuleNotFoundError: No module named 'needed_files''
    
    # from needed_files.q_env_config import q_env_config as gate_q_env_config
    
    q_env = QuantumEnvironment(gate_q_env_config)
    q_env.backend = backend
    
    ppo_agent = make_train_ppo(agent_config, q_env)

    # training_results = ppo_agent(total_updates=hyperparams["num_updates"], print_debug=True, num_prints=40)
    training_results = ppo_agent(total_updates=num_total_updates, print_debug=False, num_prints=40)
    # save_job_result({ "measurement_counts": training_results })

    print("Job completed!!!!!")

    training_results['task_summary'] = braket_task_costs.quantum_tasks_statistics()
    training_results['estimated cost'] = float(
            braket_task_costs.qpu_tasks_cost() + braket_task_costs.simulator_tasks_cost()
        )

    return training_results

#### Check if function works as expected

In [5]:
%%time

# training_results = calibrate_gate(num_total_updates=1)

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 5.25 µs


### Use a decorator to make it a Hybrid Job

In [6]:
job_name = f'cx-gate-calibration-decorator-{str(int(time.time()))}'

In [7]:
os.getcwd()

'/Users/lukasvoss/Documents/Master Wirtschaftsphysik/Masterarbeit Yale-NUS CQT/Quantum_Optimal_Control/00_AWS/04_Tests/00_Hybrid_Job'

In [8]:
needed_modules = [
    'needed_files.quantumenvironment', 
    'needed_files.helper_functions', 
    'needed_files.ppo', 
    'needed_files.q_env_config',
]


@hybrid_job(device=None, local=False, include_modules=needed_modules, input_data='config_yamls/agent_config.yaml', job_name=job_name, dependencies='requirements.txt')
def calibrate_gate_hybrid_job(num_total_updates):
    return calibrate_gate(num_total_updates)

In [9]:
job = calibrate_gate_hybrid_job(num_total_updates=10)

RuntimeError: Serialization failed for decorator hybrid job. If you are referencing an object from outside the function scope, either directly or through function parameters, try instantiating the object inside the decorated function instead.

In [ ]:
while job.state() not in job.TERMINAL_STATES:
    print(job.state())
    time.sleep(30)

print(job.state())
print(job.result())